In [ ]:
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR


import os
import numpy as np

from Stage1.modelsMultitalk.stage1_vocaset import VQAutoEncoder
from Stage1.metrics.loss import calc_vq_loss
from Stage1.base.utilities import AverageMeter
import wandb
from funciones import IntraDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cuda


In [2]:
torch.manual_seed(12)
np.random.seed(12)

In [ ]:
class Args:
    def __init__(self):
        # LOSS settings
        self.quant_loss_weight = 1.

        # NETWORK settings
        #self.arch = 'stage1_vocaset'
        self.in_dim = 39
        self.hidden_size = 1024
        self.num_hidden_layers = 6
        self.num_attention_heads = 8
        self.intermediate_size = 1536
        self.window_size = 1
        self.quant_factor = 0
        self.face_quan_num = 16
        self.neg = 0.2
        self.INaffine = False 

        # VQuantizer settings
        self.n_embed = 256
        self.zquant_dim = 64#64

        # TRAIN settings
        self.use_sgd = False
        self.sync_bn = False  # adopt sync_bn or not
        self.train_gpu = [0]
        self.workers = 10  # data loader workers
        self.batch_size = 1  # batch size for training
        self.batch_size_val = 1  # batch size for validation during training
        self.base_lr = 0.0001
        self.StepLR = True
        self.warmup_steps = 1
        self.adaptive_lr = False
        self.factor = 0.3
        self.patience = 3
        self.threshold = 0.0001
        self.poly_lr = False
        self.epochs = 200
        self.step_size = 200
        self.gamma = 0.9
        self.start_epoch = 0
        self.power = 0.9
        self.momentum = 0.9
        self.weight_decay = 0.002
        self.manual_seed = 131

        ##stage 2
        self.device = 'cuda'  # or 'cpu'
        self.dataset = 'BIWI'  # or 'multi' depending on your dataset
        self.wav2vec2model_path = 'path/to/wav2vec2model'  # path to pretrained Wav2Vec2 model
        self.feature_dim = 1024  # dimension for the feature after audio encoding
        self.vertice_dim = 31  # number of vertices * 3 (e.g., V * 3 for 3D coordinates)
        self.n_head = 8  # number of attention heads in the transformer decoder
        self.num_layers = 6  # number of layers in the transformer decoder
        self.period = 2#100  # period for positional encoding
        #self.face_quan_num = 16  # quantization levels per face/vertex
        #self.zquant_dim = 64  # dimension of the quantized latent space
        self.vqvae_pretrained_path = 'modelos-entrenados/fast-bee.pth'  # path to pretrained VQ-VAE
        self.train_subjects = 'subject1 subject2 subject3'  # space-separated list of subjects used in training
        self.motion_weight = 1.0  # weight for the motion loss
        self.reg_weight = 0.1  # weight for the regularization loss
        self.batch_size = 1#32  # batch size for training
        #self.epochs = 100  # number of training epochs
        #self.base_lr = 0.0001  # base learning rate
        self.gpu = torch.cuda.current_device()

# Instantiate the arguments
args = Args()


hidden_size = face_quan_num *   self.zquant_dim = 64


In [4]:
print(args.face_quan_num *  args.zquant_dim )
print(args.hidden_size)

1024
1024


In [ ]:
# Usage
INTRA_FOLDER = "Data/AneuxSplines/zero-root/p15/train/"
intra_dataset = IntraDataset(INTRA_FOLDER, mode="post_order") #mode es el modo al que quiero que transforme
data_loader = DataLoader(intra_dataset, batch_size=1, shuffle=True)

#dataset de validacion
VAL_FOLDER = "Data/AneuxSplines/zero-root/p15/val/"

val_dataset = IntraDataset(VAL_FOLDER, mode = "post_order")
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=True)

Dataset size: 415
Dataset size: 52


In [6]:
def validate(val_loader, model, loss_fn, epoch, cfg):
    accumulated_loss = 0
    accumulated_rec = 0
    accumulated_quant = 0
    model.eval()

    with torch.no_grad():
        for inputs in val_loader:
            
            inputs= inputs.to(device)
            if inputs.shape[1] > 1:
                out, quant_loss, info = model(inputs)
                
                # LOSS
                loss, loss_details = loss_fn(out, inputs, quant_loss, quant_loss_weight=cfg.quant_loss_weight)
                accumulated_loss += loss
                accumulated_rec += loss_details[0]
                accumulated_quant += loss_details[1]
        

        # Calculate the average loss over the entire dataset
        avg_loss = accumulated_loss / len(val_loader)
        rec_loss = accumulated_rec / len(val_loader)
        quant_loss = accumulated_quant / len(val_loader)

            

    return avg_loss, rec_loss, quant_loss


In [ ]:
wandb.login(anonymous="must")
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: paufeldman (vesselgpt) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\lab03\_netrc


True

In [8]:
wandb.init(project="vqvae-mesh", config=args)

In [9]:
def save_best_model(model, optimizer, epoch, loss, best_loss, model_save_path="best_model.pth"):
    """
    Save the model if the current loss is better than the best recorded loss.
    
    Args:
        model (torch.nn.Module): The model being trained.
        optimizer (torch.optim.Optimizer): The optimizer used in training.
        epoch (int): The current epoch number.
        loss (float): The current epoch's loss.
        best_loss (float): The best loss recorded so far.
        model_save_path (str): Path to save the best model.
    
    Returns:
        float: The updated best loss (could be the same or updated if the model improved).
    """
    if loss < best_loss:
        #print(f"Epoch [{epoch+1}], New best model found! Loss: {loss:.4f}")
        best_loss = loss
        
        # Save the model
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
        }, model_save_path)
    return best_loss

In [ ]:
# Sample training loop
num_epochs = 500000  # Define number of epochs
best_loss = float('inf')  # Initialize to a very high value

model = VQAutoEncoder(args)

model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=args.base_lr)
#scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda epoch: max(1e-4, min(1.0, epoch / 100)))
scheduler = StepLR(optimizer, step_size=args.step_size, gamma=args.gamma)

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total number of parameters: {total_params}")

rec_loss_list = []
quant_loss_list = []
pp_list = []
total_loss_list = []

wandb.watch(model, log="all")

initial_quant_loss_weight = 100.0  # Starting weight for quantization loss
final_quant_loss_weight = 2.     # Final weight for quantization loss        
decay_epochs = 300

for epoch in range(num_epochs):
    
    model.train()
    rec_loss_meter = AverageMeter()
    quant_loss_meter = AverageMeter()
    total_loss_meter = AverageMeter()
    pp_meter = AverageMeter()
    accum = 0
    for inputs in data_loader:
        
        inputs = inputs.to(device)
        if inputs.shape[1] > 1:
            out, quant_loss, info = model(inputs)
            

            #out= model(inputs) ##para entrenar sin cuantizar

            #current_quant_loss_weight = initial_quant_loss_weight - (initial_quant_loss_weight - final_quant_loss_weight) * (epoch / decay_epochs)
            # Calculate loss
            loss, loss_details = calc_vq_loss(out, inputs, quant_loss=quant_loss, quant_loss_weight=args.quant_loss_weight)
            if torch.isnan(loss):
                breakpoint()
            #loss, loss_details = calc_vq_loss(out, inputs, quant_loss, quant_loss_weight=current_quant_loss_weight, current_epoch = epoch)
            
            #############
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #accum += loss

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            for m, x in zip([rec_loss_meter, quant_loss_meter, total_loss_meter],
                        [loss_details[0], loss_details[1], loss]):
                m.update(x.item(), 1)


        

    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    avg_loss_val, rec_loss_val, quant_loss_val = validate(val_loader, model, calc_vq_loss, epoch, args)
    # Append averaged losses after each epoch
    rec_loss = rec_loss_meter.avg
    quant_loss = quant_loss_meter.avg
    total_loss = total_loss_meter.avg #aca tengo la total loss promedio de la epoch
    
    wandb.log({
        "Epoch": epoch + 1,
        "Reconstruction Loss": rec_loss,
        "Quantization Loss": quant_loss,
        "Total Loss": total_loss,
        "Total loss val": avg_loss_val,
        "Reconstruction loss val": rec_loss_val,
        "Quantization loss val": quant_loss_val,
        "Learning Rate": optimizer.param_groups[0]['lr']
    })
    # Append to respective lists
    rec_loss_list.append(rec_loss)
    quant_loss_list.append(quant_loss)

    
    best_loss = save_best_model(model, optimizer, epoch, rec_loss_val, best_loss, "models/stage1/aneux/best-model-aneux15-zero-root.pth")

    # Print learning rate and losses every 10 epochs
    if epoch % 1 == 0:
        current_lr = optimizer.param_groups[0]['lr']
        print(f"Epoch [{epoch+1}/{num_epochs}], Total Loss: {total_loss:.4f}, Rec Loss: {rec_loss:.5f}, Quant Loss: {quant_loss:.5f}, Perplexity: {pp_meter.avg:.4f}, Learning Rate: {current_lr:.6f}")
        #print(f"Epoch [{epoch+1}/{num_epochs}], Total Loss: {loss:.4f}, Learning Rate: {current_lr:.8f}")

    # Step the scheduler
    scheduler.step()
    if epoch == 280:
        # Reinitialize with a larger step size
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=1)


Total number of parameters: 100856871
Epoch [1/500000], Total Loss: 4.2408, Rec Loss: 0.19139, Quant Loss: 4.04944, Perplexity: 0.0000, Learning Rate: 0.000100
Epoch [2/500000], Total Loss: 2.2972, Rec Loss: 0.09397, Quant Loss: 2.20319, Perplexity: 0.0000, Learning Rate: 0.000100
Epoch [3/500000], Total Loss: 1.3421, Rec Loss: 0.06938, Quant Loss: 1.27268, Perplexity: 0.0000, Learning Rate: 0.000100
Epoch [4/500000], Total Loss: 0.8515, Rec Loss: 0.06097, Quant Loss: 0.79050, Perplexity: 0.0000, Learning Rate: 0.000100
Epoch [5/500000], Total Loss: 0.6081, Rec Loss: 0.05276, Quant Loss: 0.55538, Perplexity: 0.0000, Learning Rate: 0.000100
Epoch [6/500000], Total Loss: 0.4639, Rec Loss: 0.04850, Quant Loss: 0.41543, Perplexity: 0.0000, Learning Rate: 0.000100
Epoch [7/500000], Total Loss: 0.3821, Rec Loss: 0.04675, Quant Loss: 0.33539, Perplexity: 0.0000, Learning Rate: 0.000100
Epoch [8/500000], Total Loss: 0.3300, Rec Loss: 0.04316, Quant Loss: 0.28687, Perplexity: 0.0000, Learning R

KeyboardInterrupt: 